# Challenge 5 ou mini-projet de Sécurité

Boisel Jules
Xu William
Savadoux Baptiste

In [109]:
import csv
import os
import requests
import datetime
import feedparser
import dateparser
import deepl
translator = deepl.Translator(
    "nop")

In [123]:
def enlever(text):
    return text.replace("\n", "").replace("\t", "").replace("\b", "").replace("\r", "").replace("\a", "").replace("\v", "").replace("  ","").replace(" ","")

In [124]:
def New_CSV_site(emplacement , list_site):  # ecrit dans un csv (c'est juste plus simple)
    with open(emplacement, "w", newline='', encoding='utf-8') as f:
        for i in list_site:
            f.write(i)

In [125]:
liste_site_bon_atombon_rss = [] # définition d'une variable global pour pouvoir ajouter les sites dans les 2 fonctions
liste_site_bon_atom = []
def Verification_RSS(req, site):  # vérifie s'il y a du rss dans un site
    global liste_site_bon_rss
    split = enlever(req.text)
    for i in split:
        if "rss" in i.lower() and str(site + "\n") not in liste_site_bon_rss: # Vérifie s'il y a le rss ou RSS dans le html du site  # on vérifie que le site n'est pas deja dans la liste
            liste_site_bon_rss.append(site + "\n") # on met un "\n" pour retourner à la ligne lors de l'ajout du dans le csv
    New_CSV_site("Csv_Site/site_bon_rss.csv", liste_site_bon_rss) # une foi les sites ajouter dans la liste, on crée un csv pour faciliter la lecture par la suite.

def Verification_atom(req, site):  # vérifie s'il y a "atom" dans un site (même fonction qu'au-dessus mais pour atom)
    global liste_site_bon_atom
    split = req.text.split("\n")
    for i in split:
        if "atom" in i and str(site + "\n") not in liste_site_bon_atom:
            liste_site_bon_atom.append(site + "\n")
    New_CSV_site("Csv_Site/site_bon_atom.csv", liste_site_bon_atom)

In [126]:

def Connexion_site(dossier_csv):
    with open(dossier_csv, "r") as f: # on ouvre un csv en parametre afin d'en examiner plusieurs
        readCSV = csv.reader(f)
        for i in readCSV:
            print("lien en cours", i) # c'est juste pour verifier que le programme de beug pas
            try:
                req = requests.get("".join(i)) # "".join(i) permet de relier le lien du site, car sinon il est séparé et on ne peut pas le traiter # on récupère la requests du site auquel on se connecte
                if req.status_code == requests.codes.ok: # si le site répond par une réponse 200
                    Verification_RSS(req, "".join(i)) # lance les fonctions au-dessus avec en paramètre la requests au site et le lien du site pour l'ajouter dans une liste afin de l'ajouter dans un csv par la suite
                    Verification_atom(req, "".join(i))
                else:
                    print("ca marche pas : ", i) # pour voir quel site ne fonctionne pas
            except:
                print("lien dead")

In [127]:
def recuperation_des_sites():
    Connexion_site("Csv_Site/sites.csv") # on appelle 3 fois la fonction avec nos 3 fichiers csv avec nos sites
    Connexion_site("Csv_Site/cybersecurity2.csv")
    Connexion_site("Csv_Site/message.csv")
    print("Terminé")

In [ ]:
recuperation_des_sites()

lien en cours ['https://www.cert.ssi.gouv.fr/']
lien dead
lien en cours ['https://www.ssi.gouv.fr/']
lien dead
lien en cours ['https://www.zataz.com/']
lien dead
lien en cours ['https://www.usine-digitale.fr/cybersecurite/']
lien dead
lien en cours ['https://www.silicon.fr/tag/cybersecurite']
lien dead
lien en cours ['https://www.itsecurityguru.org/']
lien dead
lien en cours ['https://securityweekly.com/']
lien dead
lien en cours ['https://www.infosecurity-magazine.com/']
lien dead
lien en cours ['https://www.futura-sciences.com/tech/cybersecurite/actualites/']
lien dead
lien en cours ['https://siecledigital.fr/cybersecurite/']
lien dead
lien en cours ['https://www.lemondeinformatique.fr/securite-informatique-3.html']
lien dead
lien en cours ['https://www.rtl.fr/sujet/cybersecurite']
lien dead
lien en cours ['https://www.zdnet.fr/actualites/cybersecurite-3900046206q.htm']
lien dead
lien en cours ['https://www.cyberocc.com/sinformer/actualites-cyber/']
lien dead
lien en cours ['https://

In [128]:
def recuperation_lien_rss():  # recupere le lien du rss
    csvv = open("Csv_Site/lien_rss.csv", "w", newline='', encoding='utf-8')
    with open("Csv_Site/site_bon_rss.csv", "r") as f:
        readCSV = csv.reader(f)
        for i in readCSV:
            print ('lien en cours : ' , i)
            try:
                st = requests.get(i[-1]).text.replace("'", '"')
            except:
                continue
            balise_complet,lien = "",""
            bool_balise,balise_lien = False,False
            # temp = []
            for j in range(len(st)-1):  # on check char par char

                #Check si c'est le debut d'une balise <a ... >
                if st[j] + st[j + 1] == "<a":
                    bool_balise = True

                #check si c'est une balise fermante de <a ... > soit </a> on met bool_balise a false pour arreter de recuperer les idnex
                if st[j - 4] + st[j - 3] + st[j - 2] + st[j - 1] == "</a>":  # check si fin
                    bool_balise = False
                    if "rss" in balise_complet.lower() and "#" not in balise_complet:

                        #parfois c'est juste le /feed/ etc sans le lien devant
                        if "http" not in lien:
                            csvv.write(i[-1][:-1] + lien + "\n")
                        else:
                            csvv.write(lien + "\n")
                        # temp += lien,
                    balise_complet,lien = "",""

                ####### Dans la var lien #######
                #permet de pouvoir recuperer les index utiles Si True
                if st[j - 6] + st[j - 5] + st[j - 4] + st[j - 3] + st[j - 2] + st[j - 1] == 'href="':
                    balise_lien = True
                #Si false, ne permet de plus de recuprer les index
                if st[j] == '"' and st[-1] != "=":  # check si fin
                    balise_lien = False

                #Ecriture si balise == True
                if bool_balise:
                    #dans la balise complet
                    balise_complet += st[j]
                    if balise_lien:
                        #le lien
                        lien += st[j]
    csvv.close()

In [ ]:
recuperation_lien_rss()

In [129]:
def recuperation_atom():
    liste_line_atom = []
    with open("Csv_Site/site_bon_atom.csv", "r") as f: # boucle dans le fichier csv ou il y a les sites possedant des lien atom
        readCSV = csv.reader(f)
        for i in readCSV: # on fait tourner i dans le csv (ici i va être égale à chaque lettre du fichier)
            print ('lien en cours : ' , i)
            req = requests.get("".join(i)) # envoie une requête au site du fichier csv
            split = req.text.split("\n") # on split la chaine à chaque fois qu'il y a un retour à la ligne pour sépararer chaque ligne de l'html
            for j in split: # on fait tourner j dans le split (ici j va être égale a chaque ligne du fichier)
                if "atom" in j: # si il y a atom dans la ligne html
                    liste_line_atom.append(j + "\n") # si on trouve atom, on ajoute la ligne avec un retour à la ligne a la fin dans la liste
    with open("Csv_Site/line_atom.csv", "w", newline='', encoding='utf-8') as f: # puis on ajoute tout dans un csv
        for i in liste_line_atom:
            f.write(i)

In [94]:
recuperation_atom()

lien en cours :  ['https://www.rtl.fr/sujet/cybersecurite']
lien en cours :  ['https://thehackernews.com/']
lien en cours :  ['https://www.schneier.com/blog/archives/2022/10/regulating-daos.html']
lien en cours :  ['https://www.helpnetsecurity.com/']


KeyboardInterrupt: 

In [130]:
def recuperation_lien_atom():

    ecriture = open ("Csv_Site/lien_atom.csv", "w", newline='', encoding='utf-8')
    with open("Csv_Site/line_atom.csv", "r") as f:
        f = ''.join(f.readlines())
        # print(list(map(str,f)))
        balise_complet,balise_lien = "",""
        bool_complet,bool_lien = False,False
        # temp = []
        for i in range(len(f) - 1):
            # si on croise <a on met g true afin de pouvoir ecrire
            if f[i] + f[i + 1] == "<a":
                bool_complet = True
                # si on croise </a> on arrete l'ecriture avec g==false ou si on croise "> er que g==True pour eviter les div <div blabla ... ">
            if f[i - 4] + f[i - 3] + f[i - 2] + f[i - 1] == "</a>" or (
                    f[i - 2] + f[i - 1] == '">' and bool_complet == True):  # check si fin
                bool_complet = False
                if "atom" in balise_complet.lower() and "#" not in balise_complet:  # eviter les les #
                    # print(l.index("atom"))
                    tempo = balise_complet.index("atom")
                    # si    [pas de lettre ou de chiffre]atom[pas de lettre ou de chiffre] ou [pas de lettre ou de chiffre]atoms[pas de lettre ou de chiffre]
                    if (balise_complet[tempo - 1].isalnum() == False and balise_complet[tempo + 4].isalnum() == False) or (
                            balise_complet[tempo - 1].isalnum() == False and balise_complet[tempo + 5].isalnum() == False and balise_complet[
                        tempo + 4] == "s"):
                        # print(l)
                        # temp += balise_lien,
                        ecriture.write(balise_lien+"\n")
                        print ('lien en cours : ' , balise_lien)
                # print(l)
                # on renitialise tout
                balise_complet = ""
                balise_lien = ""
                # si on croise href on met h true afin de pouvoir ecrire
            if f[i - 6] + f[i - 5] + f[i - 4] + f[i - 3] + f[i - 2] + f[i - 1] == 'href="':
                bool_lien = True
                # si on croise " et != = on met h false afin d'arreter l'ecriture
            if f[i] == '"' and f[i + 1] != "=":  # check si fin
                bool_lien = False
            if bool_complet:  # g == True on ecrit
                balise_complet += f[i]
                if bool_lien:  # h == True on ecrit
                    balise_lien += f[i]
    ecriture.close()
    # with open ("Csv_Site/lien_atom.csv", "w", newline='', encoding='utf-8') as f:
    #     for i in temp:
    #         f.write(i+"\n")

In [ ]:
recuperation_lien_atom()

In [131]:
#fonction qui retire les balises dans des balises <description>
#par ex : <description>J'aime les aptes a <p> vanille</p>!!!</description>
#la fonction enlevera la balaise <p> et </p>
def refra(court):
    resultat=""
    des_bool_sign = True
    court = enlever(court)
    for l in range(len(court) - 3):
        if court[l] == "<":
            des_bool_sign = False
        if court[l] == ">":
            des_bool_sign = True
        if des_bool_sign and (court[l].isalnum() or court[l] not in "<>{}[]"):
            if court[l] + court[l + 1] + court[l + 2] + court[l + 3] == "http":
                resultat += " "
            resultat += court[l]
    return resultat


In [132]:
month = {'01': 'janauary',
            '02': 'february',
            '03': 'march',
            '04': 'april',
            '05': 'may',
            '06': 'june',
            '07': 'july',
            '08': 'august',
            '09': 'september',
            '10': 'october',
            '11': 'november',
            '12': 'december'}

#date_debut = datetime.datetime(2000, 10, 1)

def datee(v, date_debut):
    date = " "+v.lower().replace("-"," ").replace("/"," ").replace(",","")
    # a = " 2022-10-14T14:30:45Z".lower().replace("-"," ").replace("/"," ")

    b= ""
    #remplace les lettres par des espaces
    for i in range(len(date)-1):
        if ((date[i-1].isdigit() or date[i-1].isalpha()==False) and date[i].isalpha() and (date[i+1].isdigit() or date[i+1].isalpha()==False)) == True:
            b+=" "
        else:
            b+=date[i]
    # print(b)
    date=b
    # print(date)

    current = {"annee": "", "mois": "", "jour": ""}

    f=[]
    c= False
    b = []

    for i in range(len(date)-4):


        #check si c'est le jour/mois soit possede 2 chiffres

        if date[i-1].isalnum()==date[i+2].isalnum()==False and date[i].isdigit()==date[i+1].isdigit()==True and (date[i+2] not in ":" and date[i-1] not in ":"):
            f+=date[i]+date[i+1],

        if date[i-1].isalnum()==date[i+1].isalnum()==False and date[i].isdigit()==True and (date[i+1] not in ":" and date[i-1] not in ":"):
            f+=date[i]+date[i+1],

        #check si c'est l'année soit possede 4 chiffres
        if date[i-1].isalnum()==False and date[i].isdigit() and date[i+1].isdigit() and date[i+2].isdigit() and date[i+3].isdigit() and date[i+4].isalnum()==False and date[i+4] not in ":/-" and date[i-1] not in ":/-":
            f+=date[i]+date[i+1]+date[i+2]+date[i+3],

        #si on a des mois en lettre on converti et on insert direct dans le dictionnaire
        for j,k in month.items():
            if date[i]+date[i+1]+date[i+2] in k:
                # print(j)
                current["mois"] = j
                c=True
    # print(f)
    # print(current)
    if len(f) >0:

        #si il y a eu le mois en lettre
        if c:
            current["jour"]=f[0]
            current["annee"]=f[-1]


        #si il n'y a pas eu de mois en lettre
        else:
            # print(f)
        #Seulement si on respect le format aaaa/mm/jj
            current["annee"] = f[0]
            # current["mois"] = f[1]
            current["mois"] = f[1]
            current["jour"] = f[-1]
        #on convertit en format date
        # print(current,"-----------------------------------------")
        # print(c)

        # try:
        date_voulu = datetime.datetime(int(current["annee"]), int(current["mois"]), int(current["jour"]))
        # except:
        #     date_voulu = datetime.datetime(int(current["annee"]), int(current["jour"]), int(current["mois"]))

        # print(date_voulu)
        #on check si c'est > a date voulu
        if date_debut<date_voulu:
            return True
    else:
        print(f)
        return False

# date_voulue = datetime.datetime(2000, int("01"), int("01"))
# datee("2022-10-14T14:30:45Z",date_voulue)
# datee("Wed, 02 Nov 2022 14:11:30 +0000",date_voulue)


In [133]:
def traducteur(phrase):
    import requests, uuid, json

    # Add your key and endpoint

    #api key william azure translator creer de ma propre poche
    key = "fb8ce793c54347e19cdd06eced15d4a6"
    endpoint = "https://api.cognitive.microsofttranslator.com"

    # location, also known as region.
    # required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
    location = "francecentral"

    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'to': ['fr']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        # location required if you're using a multi-service or regional (not global) resource.
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # You can pass more than one object in body.
    body = [{
        'text': phrase
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return response[0]['translations'][0]['text']


In [134]:
 # code avec librairie feedparser


def parser_site(emplacement, mot, ecriture, date_voulue): # permet de récuperer les informations comme le titre, la description, la date et le lien de l'article qui comporte le mot recherché.
    writer = csv.writer(ecriture) # pour éviter de réécrire plusieur foi la même chose
    writer.writerow(''.join(mot)) # on ecrit le mot dans la première ligne
    writer.writerow(["title", "description", "date", "link"]) # on écrit l'entête
    with open(emplacement, "r", encoding='utf-8') as f:
        check_doublon=[]
        reader = csv.reader(f)
        for i in reader:
            print("lien en cours :", i)
            lib_feedparser = feedparser.parse("".join(i)) # permet de parser nos sites
            # print(lib_feedparser)
            for j in range(len(lib_feedparser.entries)):
                try:
                    # print(lib_feedparser.entries[j].description)
                    if lib_feedparser.entries[j].updated is not None and lib_feedparser.entries[j].description is not None and lib_feedparser.entries[j].title is not None and lib_feedparser.entries[j].link is not None :
                        if mot.lower() in (rapid:=traducteur(refra(str(lib_feedparser.entries[j].title.lower())))) or mot.lower() in (rapid2:=traducteur(refra(str(lib_feedparser.entries[j].description.lower())))): # on les mets en lower pour être sûr de la comparaison
                            data = rapid, rapid2, str(refra(lib_feedparser.entries[j].updated)) , refra(lib_feedparser.entries[j].link)
                            if datee(lib_feedparser.entries[j].updated, date_voulue) and data not in check_doublon:
                                print(data)
                                #data = traducteur(str(refra(d.entries[j].title)), target_lang="FR")), traducteur(str(refra(d.entries[j].description)), target_lang="FR")), str(d.entries[j].updated) , refra(d.entries[j].link) # on ajoute dans data le titre traduit, la derscription traduit, la date d'update, et le lien de l'article
                                writer.writerow(data) # et ensuite on les ajoute dans le csv
                                check_doublon.append(data)
                except:
                    if lib_feedparser.entries[j].updated is not None and lib_feedparser.entries[j].title is not None and lib_feedparser.entries[j].link is not None :
                        if mot.lower() in (rapid3:=traducteur(refra(str(lib_feedparser.entries[j].title.lower())))): # on les mets en lower pour être sûr de la comparaison
                            data = rapid3,"pas de description", str(refra(lib_feedparser.entries[j].updated)) , refra(lib_feedparser.entries[j].link)
                            print(lib_feedparser.entries[j].updated)
                            if datee(lib_feedparser.entries[j].updated, date_voulue) and data not in check_doublon:
                                # print(data)
                                #data = traducteur(str(refra(d.entries[j].title)), target_lang="FR")), traducteur(str(refra(d.entries[j].description)), target_lang="FR")), str(d.entries[j].updated) , refra(d.entries[j].link) # on ajoute dans data le titre traduit, la derscription traduit, la date d'update, et le lien de l'article
                                writer.writerow(data) # et ensuite on les ajoute dans le csv
                                check_doublon.append(data)
            # break

# parser_site("Csv_Site/lien_rss.csv", "veille", ecriture_rss:=open("recherche_rss.csv", "w", newline='', encoding='utf-8'), datetime.datetime(2000, 1, 1))
# ecriture_rss.close()

In [ ]:
ecriture_rss = open("recherche_rss.csv", "w", newline='', encoding='utf-8') # on affilie a ecriture l'ouverture du fichier csv où on veut ecrire les resultats
ecriture_atom = open("recherche_atom.csv", "w", newline='', encoding='utf-8') # on affilie a ecriture l'ouverture du fichier csv où on veut ecrire les resultats

boolean = True

while boolean:
    mot = str(input("Entrez le mot/thèmes que vous souhaitez")).lower() # pour faire une boucle tant qu'on sort pas pour afficher plusieurs mots
    print(mot)

    if mot == "q":
        break
    else:
        annee_voulue = int(input("Entrez l'année jusqu'à quand les articles doivent au maximum datée "))
        mois_voulue = int(input("Entrez le mois jusqu'à quand les articles doivent au maximum datée "))
        jours_voulue = int(input("Entrez le jour jusqu'à quand les articles doivent au maximum datée "))
        date_voulue = datetime.datetime(annee_voulue, mois_voulue, jours_voulue)
        parser_site("Csv_Site/lien_rss.csv", mot, ecriture_rss, date_voulue) # on l'appelle 2 fois pour différencier le rss de l'atom
        parser_site("Csv_Site/lien_atom.csv", mot, ecriture_atom, date_voulue)


ecriture_rss.close()
ecriture_atom.close()


piratage
lien en cours : ['https://www.cert.ssi.gouv.fr/feed/']
lien en cours : ['https://www.cert.ssi.gouv.fr/feed/scada/']
lien en cours : ['https://www.cert.ssi.gouv.fr/alerte/feed/']
lien en cours : ['https://www.cert.ssi.gouv.fr/cti/feed/']
lien en cours : ['https://www.cert.ssi.gouv.fr/avis/feed/']
lien en cours : ['https://www.cert.ssi.gouv.fr/ioc/feed/']
lien en cours : ['https://www.cert.ssi.gouv.fr/dur/feed/']
lien en cours : ['https://www.cert.ssi.gouv.fr/actualite/feed/']
